<img src=" " height="300" width="300"> 

# <center> R для тервера и матстата. <br>  <br> 6.2 Проверка гипотез. Бустрап и правдоподобие. </center>

Данный ноутбук является конспектом по курсу «R для теории вероятностей и математической статистики» (РАНХиГС, 2017-2018). Автор ноутбука [вот этот парень по имени Филипп.](https://vk.com/ppilif) Если у вас для него есть деньги, слава или женщины, он от этого всего не откажется. Ноутбук распространяется на условиях лицензии [Creative Commons Attribution-Share Alike 4.0.](https://creativecommons.org/licenses/by-sa/4.0/) При использовании обязательно упоминание автора курса и аффилиации. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу курса.](https://fulyankin.github.io/R_probability/) На ней можно найти другие материалы. Фрагменты кода, включенные в этот notebook, публикуются как [общественное достояние.](https://creativecommons.org/publicdomain/zero/1.0/)

---------

В прошлой тетрадке мы обсудили небольшое количество критериев для простых известных распределений. В мире осталось ещё куча неизведанного. И с этим неизведанным хочется расправляться одной левой. В этой тетрадке пойдёт речь о борьбе с неизвестностью. Будет затронуто три сюжета: 

# 1. Отношение правдоподобий 

Итак, есть небольшое количество простых известных распределений. 




И это можно. Оказыватеся, что  

$$
LR = 2\cdot(\ln L(\hat \theta_ML) - \ln L(\theta_0)) \sim \chi^2_q,
$$

где $q$ это количество ограничений. Проверка любой параметрической гипотезы благодаря методу максимального правдоподобия сводится к простому рецепту:

1. Оцениваем модель без ограничений. 
2. Оцениваем модель с ограничением, которое на неё накладывает $H_0$.
3. Смотрим как сильно отличаются правдоподобия. 

# 1. Бутсрап

__Бутстрап__ - это метод оценки статистик сложных распределений. Часто возникает необходимость проверить гипотезу о какой-то очень неудобной статистике, распределение которой неизвестно или построить для неё доверительный интервал. 

Чтобы сделать это, надо придумать как распределение этой статистики получить. Мы с вами уже знаем два способа сделать это. 

Способ номер один называется параметрическим. Мы предполагаем, что генеральная совокупность имеет какое-то распределение. Мы можем даже проверить гипотезу об этом с помощью какого-нибудь критерия. После мы на основе этого распределения можем придумать адекватную случайную величину для проверки гипотезы. Именно так мы поступали раньше. Иногда такую случайную величину придумать сложно или вовсе невозможно. Например, поди придумай адекватный критерий для медианы или моды. К тому же не очень понятно из каких соображений выбирать семейство распределений для проверки гипотезы, так как про данные ничего неизвестно. 

Способ номер два говорит следущее: давайте извлечём из генеральной совокупности какое-то количество выборок, посчитаем по ним нашу статистику, для которой доверительный интервалы мы хотим получить, а потом оценим эмпирически её выборочное распределение и на его основе получим доверительные интервалы. Этот способ применим скорее в теории, чем на практике. Если мы можем генерировать бесконечное число выборок из генеральной совокупности, то для нас незатруднительно посчитать истиное значение интересующих нас параметров.  

Второй способ подводит нас к идее бустрапа. В нашем распоряжении есть выборка. Давайте сделаем вид, что она и есть генеральная совокупность и будем извлекать из неё с повторением элементы. На основе получившихся подвыборок мы можем оценить всё, что нашей душе угодно. Поначалу такое кажется безумием, но это реально работает. И есть даже несколько теорем, которые доказывают почему это работает и как нужно правильно делать это в сложных ситуациях. 

Опишем бутсрап чуть более формально. Пусть имется выборка $X$ размера $N$. Равномерно возьмём из выборки $N$ объектов с возвращением. Это означает, что мы будем $N$ раз выбирать произвольный объект выборки (считаем, что каждый объект достаётся с одинаковой вероятность $\frac{1}{N}$), причём каждый раз мы выбираем из всех исходных $N$ объектов. Можно представить себе мешок, из которого достают шарики: выбранный на каком-то шаге шарик возвращается обратно в мешок, и следующий выбор опять делается равновероятно из того же числа шариков. Конечно же из-за возвращения среди них окажутся повторы. 

![](https://hsto.org/getpro/habr/post_images/198/14a/3f7/19814a3f7efaa3b7d1a3f8ab35889832.jpg)

Обозначим новую выборку через $X_1$. Повторяя процедуру $M$ раз, сгенерируем $M$ подвыборок $X_1, \ldots, X_M$. Теперь мы имеем достаточно большое число выборок и можем оценивать различные статистики исходного распределения. 

По изначальной выборке мы могли посчитать всего одну статистику. По сгенерированным подвыборкам мы можем посчитать $M$ статистик и увидеть как наша неизвестная статистика распределена. 

Скорее всего, вам кажется это сложным. Давайте попробуем посмотреть на конкретный пример. Возможно, станет легче. Но это неточно. 

### Проблема 

Verizon — основная региональная телекоммуникационная компания (Incumbent Local Exchange Carrier, ILEC) в западной 
части США. В связи с этим данная компания обязана предоставлять сервис ремонта телекоммуникационного оборудования 
не только для своих клиентов, но и для клиентов других локальных телекоммуникационых компаний (Competing Local Exchange Carriers, CLEC). При этом в случаях, когда время ремонта оборудования для клиентов других компаний существенно выше, чем для собственных, Verizon может быть оштрафована. 

Проверим правда ли время на ремонт своего оборудования существенно ниже чем время на ремонт других компаний.

In [55]:
library('dplyr')
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/verizon.csv', sep='\t')
# ILEC - внутренний клиент 
# CLEC - внешние клиенты 
head(df)

Time,Group
17.50,ILEC
2.40,ILEC
0.00,ILEC
0.65,ILEC
22.23,ILEC
1.20,ILEC


В выборке имеется сильный перекос в наблюдениях. 

In [56]:
sum(df$Group == 'ILEC')
sum(df$Group == 'CLEC')

[1] 1664

[1] 23

Из-за наличия такого перекоса гипотезу уместно сформулировать в терминах медиан, так как они нечувствительны к выбросам в отличие от средних. 

$$
\begin{aligned}
&H_0: \hspace{2mm} med_1 = med_2 \\
&H_1: \hspace{2mm} med_1 > med_2
\end{aligned}
$$

Беда в том, что для медиан как раз у нас нет хорошей статистики. Это весомый повод прибегнуть к бустрапу!

Давайте построим для наших медиан доверительные интервалы с помощью бутстрапа. Напишем две функции. Одна будет создавать забустрапированную выборку размера `n_samples`. 

In [61]:
get_bootstrap_samples <- function(data, n_samples){
    samples = list() # лист, куда мы будем записывать выборки
    
    for(i in 1:n_samples){
        # генерируем таблицу с повторениями соответствующую по размерам оригинальной
        # и записываем её в лист как одну из выборок и так n_samples раз 
         samples[[i]] = sample(data, size = length(data), replace=TRUE)
        }
    return(samples)
    }

In [64]:
get_bootstrap_samples (df$Time[1:5], 4) # та дам! четыре выборки внутри нашего листа :) 

[[1]]
[1]  2.40 22.23  2.40  0.65  0.00

[[2]]
[1]  0.0 17.5  2.4  0.0  0.0

[[3]]
[1]  0.65 17.50  0.00  0.00 22.23

[[4]]
[1]  0.65 22.23  2.40  2.40  0.00

Вторая функция будет искать квантили сгерированого распределения. 

In [98]:
stat_intervals <- function(stat,alpha){
    return(quantile(stat, c(alpha/2,(1-alpha/2)), name = FALSE))
    }

Отделим данные по внутренним и внешним клиентам друг от друга и построим доверительные интервалы для каждой из медиан. Будем бустрапировать $1000$ выборок. 

In [99]:
ilec = df[df$Group == 'ILEC',]
clec = df[df$Group == 'CLEC',]

# Делаем 1000 выборок для первой таблички и считаем по каждой медиану 
# функция lapply применяет функцию median к каждой выборке 
# команда as.numeric сделает листы векторами

clec_median_scores = as.numeric(lapply(get_bootstrap_samples(clec$Time, 1000), median))
ilec_median_scores = as.numeric(lapply(get_bootstrap_samples(ilec$Time, 1000), median))

Смотрим на итоговые доверительный интервалы. 

In [103]:
cat("95% confidence interval for the ILEC median repair time:", stat_intervals(ilec_median_scores, 0.05),'\n')
cat("95% confidence interval for the CLEC median repair time:", stat_intervals(clec_median_scores, 0.05))

95% confidence interval for the ILEC median repair time: 3.22 3.820125 
95% confidence interval for the CLEC median repair time: 5.79125 20.28

По аналогии найдём доверительный интервал для разности медиан. 

In [106]:
delta_median_scores = clec_median_scores - ilec_median_scores
cat("95% confidence interval for the diff median repair time:", stat_intervals(delta_median_scores, 0.05))

95% confidence interval for the CLEC median repair time: 2.04975 16.6805

Оценим вероятность того, что разность больше нуля. 

In [112]:
sum(delta_median_scores > 0)/length(delta_median_scores)

[1] 0.999

Кажется, команию пришло время оштрафовать. 

# 2. Генерируем распределение для среднего 

